In [2]:
%pip install python-terrier 
%pip install --upgrade python-terrier


Note: you may need to restart the kernel to use updated packages.
Requirement already up-to-date: python-terrier in ./scrapy_env/lib/python3.8/site-packages (0.10.0)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyterrier as pt
import pandas as pd

if not pt.started():
  pt.init()


/home/christian/Desktop/information_retrieval/scrapy_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
df = pd.read_json("./results_esa.json")
ids = []
for i in range(len(df)):
    ids.append(f"d{i}")
df['docno'] = ids
df['text'] = df['mission_name'] + df['mission_description'] + df['mission_page']
df.drop(['mission_name','mission_description','mission_page'],axis=1,inplace=True)
df.head(5)

,docno,text
0,d0,Aeolus Aeolus was the first satellite mission ...
1,d1,Hera As part of the world’s first test of aste...
2,d2,GOCE ESA's Earth Explorer GOCE mission was ded...
3,d3,"Φ-sat Ф-sat-1, an enhancement of the Federated..."
4,d4,Webb The James Webb Space Telescope (Webb) is ...


In [104]:
df_2 = pd.read_json("./results_spaceflight_no3.json")
ids = []
for i in range(len(df),len(df_2)+len(df)):
    ids.append(f"d{i}")
df_better = pd.DataFrame()
df_better['docno'] = ids
df_better['text'] = f"{df_2['title']} \n {df_2['date']}  \n {df_2['description']} \n {df_2['body'][:-34]}".lower()
# df_better.info()


In [105]:
texts = []
for i in range(df_2.shape[0]):
    prova = df_2.iloc[i, :]
    texts.append((prova['title'] + " " + prova['description'] + " " + prova['body'][:-34] + " " + prova['date'].strftime("%d-%B-%Y")).lower())
prova['date'].strftime("%d %B %Y")

'13 April 2018'

In [109]:
df_better['text'] = texts
df_better.info
df_better = pd.concat([df,df_better])

In [3]:
indexer = pt.DFIndexer("./index_3docs", overwrite=True)
index_ref = indexer.index(df["text"], df["docno"])
index_ref.toString()

'./index_3docs/data.properties'

In [4]:
index = pt.IndexFactory.of(index_ref)

#lets see what type index is.
type(index)

jnius.reflect.org.terrier.structures.Index

In [5]:
print(index.getCollectionStatistics().toString())

Number of documents: 28
Number of terms: 571
Number of postings: 1119
Number of fields: 0
Number of tokens: 1393
Field names: []
Positions:   false



In [6]:
word = 'rocket'
pointer = index.getLexicon()[word]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

ID(6) TF(1) doclen=48
ID(17) TF(1) doclen=74


In [7]:
br = pt.BatchRetrieve(index, wmodel="BM25") #Query with a single word
br.search("rocket")
# print(df_better['text'][2])

,qid,docid,docno,rank,score,query
0,1,6,d6,0,3.455721,rocket
1,1,17,d17,1,2.839732,rocket


In [116]:
queries = pd.DataFrame([["q1", "stars"]], columns=["qid", "query"])
br.transform(queries)

,qid,docid,docno,rank,score,query
0,q1,681,d681,0,7.127961,stars
1,q1,281,d281,1,7.079655,stars
2,q1,683,d683,2,7.069804,stars
3,q1,3068,d3068,3,7.007452,stars
4,q1,582,d582,4,6.975908,stars
...,...,...,...,...,...,...
892,q2,2489,d2489,584,1.215587,polar
893,q2,2313,d2313,585,1.122792,polar
894,q2,3129,d3129,586,1.058427,polar
895,q2,2003,d2003,587,1.011544,polar


In [101]:
bm25 = pt.BatchRetrieve(index, num_results = 15, wmodel="BM25")
queries = pd.DataFrame([["q1", "stars solar system"],["q2", "solar system"]], columns=["qid", "query"])
results = bm25.transform(queries)
results

,qid,docid,docno,rank,score,query
0,q1,554,d554,0,8.082120,stars solar system
1,q1,954,d954,1,7.938552,stars solar system
2,q1,3296,d3296,2,7.863355,stars solar system
3,q1,2318,d2318,3,7.691179,stars solar system
4,q1,2901,d2901,4,7.631998,stars solar system
5,q1,3251,d3251,5,7.480179,stars solar system
6,q1,652,d652,6,7.457715,stars solar system
7,q1,767,d767,7,7.394576,stars solar system
8,q1,3509,d3509,8,7.352769,stars solar system
9,q1,927,d927,9,7.347446,stars solar system
